In [8]:
#Imports
import numpy as np
import sympy as sym
from matplotlib import pyplot as plt
from scipy.integrate import quad
from scipy.integrate import dblquad
import scipy.optimize as so
import scipy.special as ss
from scipy.special import ellipe
from scipy.special import ellipk

In [9]:
#Constants
Mbh = 2.7e9                                                 #mass of the central black hole in (solar mass)
G = 4.300e-6                                                #gravitational constant (kpc/solar mass*(km/s)^2)
h = 8.9                                                     #radial scale-length (kpc)
rho00 = 0.31e9                                              #prefactor that will cancel
epsdisk = 5.0                                               #from Noordermeer's paper
absmag = -22.02                                             #absolute magnitude 
magsun = 4.42                                               #absolute magnitude of the sun
z0 = 0.2*h                                                  #half-thickness (kpc)
R = 4*h                                                     #cut-off radius (kpc)
d = 0.2*h                                                   #cut-off length upper limits (kpc)
L0 = np.power(10, (0.4*(magsun-absmag)))                    #Absolute Magnitude to luminosity
n = 2.7                                                     #concentration parameter that describes the curvature of the profile in a radius-magnitude plot, n=4 is de Vaucoileurs profile
re = 2.6                                                    #1kpc
ups = 2.8                                                   #mass-to-light ratio (from Rotation Curves of Sersic Bulges paper)
q = 0.33                                                    #intrinsic axis ratio
i = 45*(np.pi/180)                                          #inclination angle
L = 3.27e10                                                 #luminosity

In [10]:
#Variables
x = np.linspace(1, 10, 100)                        #x from/to and line smoothness

In [11]:
#############BLACK HOLE####################

#equation for orbital velocity
def vbh(r, G, Mbh):
    return np.sqrt((G*Mbh)/r)

In [ ]:
#############DISK##################

###########################################Intermediate Functions#####################################################

#Define Initial Functions
def x(r,u,xi):
    return ((r**2)+(u**2)+(xi**2))/(2*r*u)
def px(r,u,xi):
    return x(r,u,xi)-(np.sqrt((x(r,u,xi)**2)-1))

#Density Piecewise Function
def rho0(r, R, h, d):                                      
    condlist = [r <= R, (r > R) & (r <= (R+d)), r > (R+d)]
    funclist = [lambda r: rho00*np.exp(-r/h), lambda r: rho00*np.exp(-R/h)*(1-((r-R)/d)), lambda r: 0]
    return np.piecewise(r, condlist, funclist)
#Partial Derivative of rho(u,xi)
def durho0(r, R, h, d):                                  
    condlist = [r <= R, (r > R) & (r <= (R+d)), r > (R+d)]
    funclist = [lambda r: -(1/h)*rho00*np.exp(-r/h), lambda r: -(1/d)*rho00*np.exp(-R/h), lambda r: 0]
    return np.piecewise(r, condlist, funclist)

#Disk Density Distribution
rho_rz = lambda r,z: rho0(r, R, h, d)*(np.power((sym.cosh(z/z0)).evalf(), (-2)))
drho_rz = lambda r,z: durho0(r, R, h, d)*(np.power((sym.cosh(z/z0)).evalf(), -2))
#Complete Elliptic Integral
K = lambda r,u,xi: ellipk(px(r,u,xi)) - ellipe(px(r,u,xi))
#Inner Function (3D)
f = lambda r,u,z: u*drho_rz(u, z)*(2*K(r,u,z))/(np.pi*np.sqrt(r*u*px(r,u,z)))
#Integrate Function
f3 = lambda z,r,u: f(r,u,z)
intf = lambda r,u: quad(f3, 0, np.inf, args=(r,u,))[0]
#integrate outer function
intf3 = lambda u,r: intf(r,u)
intintf = lambda r: quad(intf3, 0.1, 125, args=(r,))[0]
#mass of disk
rho_rz_r = lambda z,r: rho_rz(r,z)*r
Mdblintrho = dblquad(rho_rz_r,0,125,-125,125)
#epsdisk = Mdblintrho/L0
pref = epsdisk*(L0/Mdblintrho)
#multiplying by epsylon
F = lambda r: 4*np.pi*G*intintf(r)*pref

######################################################################################################################
#Disk Velocity
rd = np.linspace(0.1, 125, num=100)
Fv = np.vectorize(F)
vd = np.sqrt(-rd*Fv(rd)) 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.


In [ ]:
#############BULGE###############

#Gamma Function
f = lambda x: ss.gammainc(2*n,x)*ss.gamma(2*n)-0.5*ss.gamma(2*n)
root = so.brentq(f,0,500000,rtol=0.000001,maxiter=100) #come within 1% of exact root within 100 iterations
r0 = re/root**n


########################################Intermediate Equations#########################################

f = lambda x,m: ((np.exp(-np.power(x/r0, (1/n))))*(np.power(x/r0, ((1/n)-1))))/(np.sqrt((x**2)-(m**2)));
g = lambda m: quad(f, m, np.inf,args=(m,))[0]
I0 = (L*(root**(2*n)))/(((re**2)*2*np.pi*n)*ss.gamma(2*n))
C = (4*G*q*ups*I0)/(r0*np.float(n))*(np.sqrt((np.sin(i)**2)+(1/(q**2))*(np.cos(i)**2)))
e2 = 1-(q**2)

#integrate outer function
h = lambda m,r: C*g(m)*(m**2)/(np.sqrt((r**2)-((m**2)*(e2))))

y = np.zeros(np.shape(x))
hr = lambda m: h(m,r)
for j,r in enumerate(x):
    y[j] = quad(h, 0, r,args=(r,))[0]

#######################################################################################################

#Final Equation
vb = np.sqrt(y)

In [ ]:
#Run necessary code from existing notebooks.
#Would like to find a way to do all imports and constants once, rather than having to set them within each script.
#Primarily the imports would be nice to skip; variables take next to no time.

#%run scripts/bulge.py
#%run scripts/disk.py
#%run scripts/DMhalo.py
#%run scripts/blackhole.py

In [ ]:
#Set remaining necessary variables

#Will not work until necessary code is determined and set up.

r0 = np.linspace(0, 10, 20)
vtotal = lambda r: np.sqrt(vcdm(r)**2+vd**2+vbh(r)**2+vb**2)
vtotalv = np.vectorize(vtotal)

In [ ]:
#Plot

#Will not work until necessary code is determined and set up.

fig = plt.figure(figsize=(9.0,8.0))                #size of the plot
ax = plt.axes()
ax.grid(True)
ax.yaxis.grid(True,which='minor',linestyle='--')

plt.plot(r0, vtotalv(r0), linestyle='solid', label='Velocity')          

plt.axis('tight');                                 #xmin, xmax, ymin, ymax
                                                   #or use: plt.xlim(10, 0) plt.ylim(1.2, -1.2);
                                                   #or to tighten the bounds: plt.axis('tight')
                                                   #or use: plt.axis([0, 15000, 0.5e6, 2.5e6])

fig.suptitle('Total Rotation Curve NGC5533', fontsize=18)
ax.set(title='NGC 5533')              #labeling axes
ax.set_xlim(0,10)
ax.set_xlabel('r (kpc)', fontsize=14)
ax.set_ylabel('v (km/s)', fontsize=14)
ax.xaxis.set_tick_params(labelsize=14)
ax.yaxis.set_tick_params(labelsize=14);

plt.legend(fontsize=16);                                   #adding legend
plt.savefig('total.png')